# 4. LME銅3M先物 - モデル比較分析

## 概要
LME銅3M先物の価格予測に向けて、複数の機械学習モデルを比較分析します。

### 比較モデル
1. Random Forest Regressor
2. XGBoost Regressor
3. Support Vector Regression
4. ARIMAモデル
5. Linear Regression
6. Ridge Regression

### 評価指標
- RMSE（Root Mean Square Error）
- MAE（Mean Absolute Error）
- R-squared
- 方向性精度
- シャープレシオ

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox

import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# データベース接続
engine = create_engine('postgresql://Yusuke@localhost:5432/lme_copper_db')

print("=== LME銅3M先物モデル比較分析システム ===")
print("データベース接続完了")

## 1. データ読み込みと特徴量作成

In [ ]:
# LME銅3M先物データの読み込み
query = """
SELECT 
    date,
    m3_price as price,
    m3_volume as volume,
    m1_price,
    m2_price,
    m4_price,
    m5_price,
    m6_price
FROM lme_copper_futures
WHERE m3_price IS NOT NULL
ORDER BY date
"""

df = pd.read_sql_query(query, engine)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

print(f"データ期間: {df.index.min()} - {df.index.max()}")
print(f"総レコード数: {len(df):,}")

df.head()